In [1]:
import torch
from torch import nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from fastai.vision.all import URLs
from batchflow import Pipeline, B, C, D, F, V, W
from batchflow.models.torch import EncoderDecoder
from batchflow.models.torch.layers import ConvBlock
from batchflow.models.torch.layers.modules import ASPP
from batchflow.opensets import PascalSegmentation

from train_module import training_functions

GRAPH_PATH = "./data/graphs/"

In [2]:
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_VISIBLE_DEVICES=3


In [3]:
dataset = PascalSegmentation(bar='n')

In [4]:
in_channels = 3
out_channels = 3
device = "cuda:3"

batch_size = 64
epoch_num = 5

IMAGE_SHAPE = (3, 160, 160)

In [25]:
downsample_depth = 2
in_channels = 3
model_config = { 
    'device': device,
    'inputs/images/shape': (3, 160, 160), # can be commented
    'initial_block/inputs': 'images', # can be commented   
    
    'initial_block': {
        'layout': 'cna cna',
        'strides': [2, 1],
        'filters': [32, 64]
    },

    'body/encoder': {
        'num_stages': downsample_depth,
        'order': ['downsampling', 'skip']
    },    
    'body/encoder/downsample': {
        'layout': 'R' + 'wnacna' * 3 + '|',
        'filters': 'same',
        'strides': [[1, 1, 1, 1, 2, 1]] * downsample_depth,
        'kernel_size': [[3, 1] * 3] * downsample_depth,
        'dilation_rate' : 2,
        'branch': {
            'layout': 'cn', 
            'kernel_size': 1,
            'filters': 'same',
            'strides': 2
        },
    },      

    'body/embedding': {
        'base' : ASPP
    },  

    'body/decoder': {
        'num_stages': downsample_depth,
        'order': ['block', 'combine', 'upsampling'] 
    },

    'body/decoder/blocks': {
        'layout': 'cna',
        'kernel_size': 1,
        'filters': 256
    },

    'body/decoder/upsample': {
        'layout': 'b',
        'scale_factor': 2
    },
    'body/decoder/combine': {
        'op': 'concat',
    },
    
    
    'head':{
        'layout': 'cna b',
        'scale_factor': 2,
        'filters': in_channels,
        'activation': 'sigmoid'
    },

    'loss': 'mse',
    'optimizer': 'Adam'
}

In [26]:
def squeeze(array):
    return np.squeeze(array)

In [27]:
# squeeze and process

train_pipeline = (dataset.train.p
                .init_variable('loss_history', [])
                .init_model('dynamic', EncoderDecoder, 'deeplab',
                            config=model_config)
                .resize(size=(160, 160), src=['images', 'labels'], dst=['images', 'labels'])
                .to_array(channels='first', src=['images', 'labels'], dst=['images', 'labels'])              
                .print(B.images.shape, B.labels.shape)
                #.squeeze(B.labels, save_to=B.labels)
                .print(B.images.shape, B.labels.shape)
                .train_model('deeplab', B.images, B.labels, 
                             fetches='loss', save_to=V('loss_history', mode='a'), use_lock=True)
)

In [28]:
train_pipeline.next_batch(3)

(3, 3, 160, 160) (3, 1, 160, 160)
(3, 3, 160, 160) (3, 1, 160, 160)


RuntimeError: Sizes of tensors must match except in dimension 2. Got 1 and 20 (The offending index is 0)

In [ ]:
train_pipeline.run(64, n_epochs=2, bar='n', shuffle=True)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(train_pipeline.v('loss_history'))
plt.xlabel("Iterations"), plt.ylabel("Loss")
plt.show()

In [ ]:
pipeline = (dataset.test.p
                .crop(shape=(160, 160), origin='center')
                .to_array(channels='first', dtype=np.float32)
                .multiply(1./255)
                .init_variable('metrics')
                .init_variable('inferred_masks')
                .import_model('deeplab', train_pipeline)
                .predict_model('deeplab', fetches='predictions', feed_dict={'x': B('images')},
                               save_to=V('inferred_masks'))
                .gather_metrics('mask', targets=B('images'), predictions=V('inferred_masks'),
                                fmt='proba', axis=-1, save_to=V('metrics', mode='u'))
            )

In [ ]:
pipeline.run(64, bar='n')

In [ ]:
batch = pipeline.next_batch(3)
outputs = pipeline.v('inferred_masks')[:3]

In [ ]:
type(outputs[0][0,0,0])

In [ ]:
type(batch.images[0][0,0,0])

In [ ]:
batch = pipeline.next_batch(3)
outputs = pipeline.v('inferred_masks')[:3]
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(8, 8), subplot_kw={'xticks': [], 'yticks': []})

for picture_num, ax in enumerate(axs.flat):
    if picture_num<3:        
        ax.imshow(np.moveaxis(batch.images[picture_num], 0, -1))
    else:
        ax.imshow(np.moveaxis(outputs[picture_num-3].astype('float64'), 0, -1))


plt.tight_layout()
plt.show()